Exploring the $\chi^2$ distribution
===

Start with some model to generate random data.  <br> 
We'll use $1+\frac{sin(x)}{x+1}$ over the range $0\leq x<15$ <br>
Generate $N$ random numbers according to this distribution and make a histogram of the results using 30 bins. 

An easy way to generate random numbers according to an arbitrary distribution is to use rejection sampling (here's a great [decription  of the technique](https://matthewfeickert.github.io/Statistics-Notes/notebooks/simulation/Rejection-Sampling-MC.html) )

1) Make a histogram of your random distribution.  Verify that the the numbers you generated agree with the shape of the function.  Generate at least 5000 points and plot your data histogram with errorbars.  You can use numpy+matplotlib+etc or pyROOT or a mix.  Think about how to appropriately normalize your function to overlay it with the data.  The normalization depends on the number of bins, the bin range and the number of points thrown.  You may find it easier to first nromalize the area of your function to 1.0, thus turning it into a proper PDF.  

In [1]:
# your work goes here

2) Repeat the above experiment at least 1000 times.  For each experiment calculate the $\chi^2$ of your data with respect to your (properly normalized) model.  Plot (histogram) the $\chi^2$ distribution and compare it to the functional form of the $\chi^2(ndof=30)$ distribution.  You can find the form of the $\chi^2(ndof)$ PDF function in many places and implement it using the Gamma function or use a premade function, eg:
* [wikipedia](https://en.wikipedia.org/wiki/Chi-squared_distribution)
* [scipy.stats.chi2](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html#scipy.stats.chi2)

In [ ]:
# your work goes here

**Only required for Phys5630**

3) Modify your code above to perform the following study.  <br>
Repeat (2), generating 10k random points for each experiment.  But calculate the reduced $\chi^2$, eg $\chi^2/ndof$, for each experiment.  For your 1000 experiments, calculate the mean value of the reduced $\chi^2$ and its standard deviation.  Repeat this for nbins = $ndof = 30, 50, 75, 100$  Plot your mean values with errorbars vs $ndof$.  How does this compare to expectations?

In [ ]:
# your work goes here